In [3]:
import xmltodict as xd
import pandas as pd
import os, glob
import mio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [4]:
def get_model_list(kml_path:str)->pd.DataFrame:
    " read kml file, get anchor coordinates for every model"
    mio.check_types(get_model_list, locals())
    with open(kml_path) as fin:
        s = fin.read()
    kmld =  xd.parse(s)
    rows = []
    for pm in kmld['kml']['Document']['Placemark'][0:]:
        model = pm['Model']
        row = {}
        row['model'] = os.path.basename(model['Link']['href'])
        sLon = model['Location']['longitude']
        sLat = model['Location']['latitude']
        row['swiss_lon'] = float(sLon)
        row['swiss_lat'] = float(sLat)
        row['swiss_x'], row['swiss_y'] = mio.wgs_swiss(sLon, sLat)
        row['geometry'] = Point(row['swiss_x'], row['swiss_y'])
        rows.append(row)
    df = pd.DataFrame(rows)
    df = df.set_index('model')
    return df

In [5]:
def get_point_lists(dae_file:str) -> dict:
    " get point and vertex list for one model"
    with open(dae_file) as fin:
        s = fin.read()
    d = xd.parse(s)
    
    geometries = d['COLLADA']['library_geometries']['geometry']
    
    res = {}
    for geo in geometries:
        geo_id = geo['@id']

        # float list
        float_list = geo['mesh']['source']['float_array']['#text'].split()
        float_list = [float(f) for f in float_list]

        # vertex list
        vert_list = geo['mesh']['triangles']['p'].split()
        vert_list = [int(n) for n in vert_list]
        # write result
        res[geo_id] = (float_list, vert_list)
    return res

In [6]:
def append_geos(dic:dict):
    " combine the v_lists and f_lists in the dictinary to one "
    v_list = []
    f_list = []
    for key in dic:
        offset = len(v_list) // 3
        v_list = v_list + dic[key][0] 
        f_list = f_list + [n+offset for n in dic[key][1]]
    return v_list, f_list

In [7]:
def write_obj(v_list:list, f_list:list, lon:float, lat:float, dest:str):
    " write v_list and f_list to an obj text 3d file"
    fout = open(dest, 'w')
    fout.write(f'g {lon} {lat}\n')
    
    va = np.array(v_list)
    va = va.reshape(len(v_list)//3, 3)
    for row in va:
        s = f"v {row[0]} {row[1]} {row[2]}\n"
        fout.write(s)
        
    fa = np.array(f_list)
    fa = fa.reshape(len(f_list)//3, 3)
    for row in fa:
        s = f"f {row[0]+1} {row[1]+1} {row[2]+1}\n"
        fout.write(s)       
    
    fout.close()

In [8]:
def make_obj(dae_path:str, dest:str, lon:float, lat:float):
    " full proecess of converting .dae to .obj file"
    dic = get_point_lists(dae_path)
    v_list, f_list = append_geos(dic)
    write_obj(v_list, f_list, lon, lat, dest)

In [9]:
def convert_kml(kml_path:str, dest_folder:str):
    prefix = mio.file_title(kml_path) + '_'
    mod_list = get_model_list(kml_path)
    i = 0
    for ind, row in mod_list.iterrows():
        i += 1
        mio.show_perc(i, len(mod_list), 100)
        dae_path = os.path.join(os.path.dirname(kml_path), 'models', ind)
        dest = dest_folder + "/" + prefix + mio.file_title(dae_path) + '.obj'
        try:
            make_obj(dae_path, dest, row.swiss_lon, row.swiss_lat)
        except:
            print('error in conver_kml()', kml_path)
    return i

In [10]:
convert_kml(r"C:\Buildings2.0\KML\1091-41\1091-41.kml", 'obj')

0.76% 1.52% 2.27% 3.03% 3.79% 4.55% 5.31% 6.07% 6.82% 7.58% 8.34% 9.1% 9.86% 10.62% 11.37% 12.13% 12.89% 13.65% error in conver_kml() C:\Buildings2.0\KML\1091-41\1091-41.kml
14.41% 15.17% 15.92% 16.68% 17.44% 18.2% 18.96% 19.71% 20.47% 21.23% 21.99% 22.75% 23.51% 24.26% 25.02% 25.78% 26.54% 27.3% 28.06% 28.81% 29.57% 30.33% 31.09% 31.85% 32.61% 33.36% 34.12% 34.88% 35.64% 36.4% 37.15% 37.91% 38.67% 39.43% 40.19% 40.95% 41.7% 42.46% 43.22% 43.98% 44.74% 45.5% 46.25% 47.01% 47.77% 48.53% 49.29% 50.05% 50.8% 51.56% 52.32% 53.08% 53.84% 54.6% 55.35% 56.11% 56.87% 57.63% 58.39% 59.14% 59.9% 60.66% 61.42% 62.18% 62.94% error in conver_kml() C:\Buildings2.0\KML\1091-41\1091-41.kml
63.69% 64.45% 65.21% 65.97% 66.73% 67.49% 68.24% 69.0% 69.76% 70.52% 71.28% 72.04% 72.79% 73.55% 74.31% 75.07% 75.83% 76.58% 77.34% 78.1% 78.86% 79.62% 80.38% 81.13% 81.89% 82.65% 83.41% 84.17% 84.93% 85.68% 86.44% 87.2% 87.96% 88.72% 89.48% 90.23% 90.99% 91.75% 92.51% 93.27% 94.02% 94.78% 95.54% 96.3% 97.06% 97.82%

13188